# Given the input csv file, scrap the genre for each song and add them into column as a string

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
def get_genre_spotify(artist_name, sp):
    try:
        result = sp.search(artist_name)
        track = result['tracks']['items'][0]

        artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
        genre_list = artist["genres"]
        tags = ''
        for genre in genre_list:
            tags = tags + genre + ','
        tags = tags.rstrip(",")
        print(tags)
    except:
        pass
    return tags

In [3]:
# scrap the genre from last.fm. However, if (very rare change) that there is no genre
# available in last.fm, we will use tge general general of the artist from spotify instead
def get_genre(track_name, artist):
    client_id = '90d4bf0d722b4e6892fedd3eb7dea15d'
    client_secret = 'b85dd00354944ae4a37e1b1aa7871f5d'
    client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    
    headers = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv:42.0) Gecko/20100101 Firefox/42.0' }
    link = f'https://www.last.fm/music/{artist}/_/{track_name}'
    try:
        req = requests.get(link, headers=headers, timeout=None)
        soup = BeautifulSoup(req.content, features="html.parser")
        tags = ""
        for tag in soup.find_all(attrs={'class':'tag'}):
            href = tag.find('a').get('href')
            href = href.replace("/tag/", "")
            tags += href+","
        print(track_name)
        
        tags = tags.rstrip(",")
        if tags == '':
            print('spotify')
            tags = get_genre_spotify(track_name, sp)
            
        return tags
    except Exception as e:
        print("not found")
        print(e)

In [8]:
def label_genre(inputfile, outputfile):
    data = pd.read_csv(inputfile)
    #data = data.head(10)
    data['Genre'] = data.apply(lambda x: get_genre(x['Track Name'], x['Artist']), axis=1)
    data.to_csv(outputfile,index = False)
    return data

In [9]:
inputfile = "song_data_combined.csv"
outputfile = "song_data_combined_genre.csv"
data = label_genre(inputfile, outputfile)
data

track name:  All I Want for Christmas Is You , artist:  Mariah Carey
track name:  Despacito (Featuring Daddy Yankee) , artist:  Luis Fonsi
track name:  I Don't Care (with Justin Bieber) , artist:  Ed Sheeran
track name:  7 rings , artist:  Ariana Grande
track name:  SAD! , artist:  XXXTENTACION
track name:  Last Christmas , artist:  Wham!
track name:  Señorita , artist:  Shawn Mendes
track name:  Shape of You , artist:  Ed Sheeran
not found
HTTPSConnectionPool(host='www.last.fm', port=443): Max retries exceeded with url: /music/Lana%20Del%20Rey/_/In%20My%20Feelings (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000018E8EB19160>: Failed to establish a new connection: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions'))
track name:  thank u, next , artist:  Ariana Grande


,Track Name,Artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Label,Genre
0,All I Want for Christmas Is You,Mariah Carey,0.336,0.627,7,-7.463,1,0.0384,0.1640,0.000000,0.0708,0.350,150.273,241107,4,1,"christmas,pop,xmas,mariah+carey,pop,rnb,female..."
1,Despacito (Featuring Daddy Yankee),Luis Fonsi,0.660,0.786,2,-4.757,1,0.1700,0.2090,0.000000,0.1120,0.846,177.833,228200,4,1,"latin,pop,daddy+yankee,meme,2017,latin,pop,lat..."
2,I Don't Care (with Justin Bieber),Ed Sheeran,0.798,0.675,6,-5.041,1,0.0442,0.0912,0.000000,0.0894,0.842,101.956,219947,4,1,"pop,2010s,2019,no+6+collaborations+project,aco..."
3,7 rings,Ariana Grande,0.725,0.321,1,-10.744,0,0.3230,0.5780,0.000000,0.0884,0.319,70.142,178640,4,1,"trap,rap,pop,rnb,pop,rnb,female+vocalists"
4,SAD!,XXXTENTACION,0.740,0.613,8,-4.880,1,0.1450,0.2580,0.003720,0.1230,0.473,75.023,166568,4,1,"hip-hop,emo,emo+rap,rap,hip+hop,hip-hop,cloud+..."
5,Last Christmas,Wham!,0.735,0.478,2,-12.472,1,0.0293,0.1890,0.000002,0.3550,0.947,107.682,262960,4,1,"christmas,80s,pop,xmas,pop,80s,british"
6,Señorita,Shawn Mendes,0.759,0.540,9,-6.039,0,0.0287,0.0370,0.000000,0.0945,0.750,116.947,190960,4,1,"pop,latin,camila+cabello,latin+pop,pop,canadia..."
7,Shape of You,Ed Sheeran,0.825,0.652,1,-3.183,0,0.0802,0.5810,0.000000,0.0931,0.931,95.977,233713,4,1,"pop,dancehall,2017,best+of+2017,acoustic,briti..."
8,In My Feelings,Lana Del Rey,0.540,0.674,2,-6.351,1,0.0802,0.0553,0.060800,0.1120,0.309,89.933,238719,4,1,None
9,"thank u, next",Ariana Grande,0.724,0.647,1,-5.642,1,0.0658,0.2800,0.000000,0.1020,0.435,106.960,207333,4,1,"pop,rnb,pete+fracassado,pop,rnb,female+vocalists"
